##Описание решения

Исполльзовалось два алгоритма:

    1) Деревья (sklearn.ensemble.RandomForestClassifier)
    2) Логистическую регрессию (sklearn.linear_model.LogisticRegression)

И деревья и логистическая регрессия обучались на следующих признаках:
    * количество слов в тексте вопроса (получено с помощью nltk), так же квадрат этого количества
    * количество слов в заголовке и в тегах (получено с помощью nltk), так же квадрат этого количества
    * количество предложений в тексте вопроса (получено с помощью nltk), так же квадрат этого количества
    * длина текста
    * длина заголовка и тегов
    * момент создания вопроса
    * момент создания аккаунта автора вопроса
    * темпы роста репутации автора (как репутация автора / разность создания вопроса и создания аккаунта автора)
    * репутация автора и логарифм репутации автора
    * для некторых слов подсчитывалось сколько раз они встретились в тесте и в заголовке ('    ' - сиволизирует количество строчек кода в вопросе, '\n' - количество абзацев...)
    
Логистическая регрессия(кроме вышеперечисленных) также обучалась на следующих признаках:
    * tfidf по тексту вопроса
    * tfidf по тексту заголовка и тегов вопроса
    * tfidf по тексту первого абзаца вопроса
    * посимвольный tfidf по тексту вопроса
    * посимвольный tfidf по тексту заголовка и тегов вопроса

Далее два алгоритма линейно конкатенировались.

In [2]:
import pandas as pd
import numpy as np
import re
import time
import datetime
import nltk
import math
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
train_path = 'real-train-contest.csv'
test_path = 'test-contest.csv'
real_test_path = 'test-contest-second.csv'
data = pd.read_csv(train_path)
datatest = pd.read_csv(test_path)
datarealtest = pd.read_csv(real_test_path)

In [3]:
def get_time(x):
    try:
        return time.mktime(datetime.datetime.strptime(x, '%m/%d/%Y %H:%M:%S').timetuple())
    except:
        return time.mktime(datetime.datetime.strptime(x, '%Y-%m-%d').timetuple())
def make_new_features(data):   
    ls = []
    for ind,row in data.iterrows():        
        s = str(row.Tag1) + ' ' + str(row.Tag2) + ' ' + str(row.Tag3) + ' ' + str(row.Tag4) + ' ' + str(row.Tag5) + ' ' + str(row.Title)
        ls.append(s)
    data['Tags'] = ls 
    data['CntWord'] = data.BodyMarkdown.apply(lambda x: len(nltk.word_tokenize(re.sub('\W+', ' ', x.lower()))))
    data['CntTitleWord'] = data.Title.apply(lambda x: len(nltk.word_tokenize(re.sub('\W+', ' ', x.lower()))))
    data['CntSentance'] = data.BodyMarkdown.apply(lambda x: len(nltk.sent_tokenize(x.lower())))
    data['TextLength'] = data.BodyMarkdown.apply(lambda x: len(x.split()))
    data['HeaderLength'] = data.Title.apply(lambda x: len(x.split()))
    data['PostCreation'] = data.PostCreationDate.apply(lambda x: get_time(x))
    data['OwnerCreation'] = data.OwnerCreationDate.apply(lambda x: get_time(x))
    data['PostAge'] = data.PostCreation - data.OwnerCreation
    data['PostAge'] = data.PostAge.apply(lambda x: max(x, 1))
    data['FirstIndent'] = data.BodyMarkdown.apply(lambda x: x[: x.find('\n')])
    data['TempInc'] = data.ReputationAtPostCreation / data.PostAge
    data['NormReputation'] = data.ReputationAtPostCreation.apply(lambda x: math.log(x + 100))
    data['NormCntWord'] = data.CntWord * data.CntWord
    data['NormCntTitleWord'] = data.CntTitleWord * data.CntTitleWord
    data['NormCntSentance'] = data.CntSentance * data.CntSentance
    
make_new_features(data)
make_new_features(datatest)
make_new_features(datarealtest)

In [4]:
print('!!!')
data_train = data
data_test = datatest
data_real_test = datarealtest
feature_set = ['ReputationAtPostCreation', 'OwnerUndeletedAnswerCountAtPostTime', 'TextLength', 'PostCreation', 
               'OwnerCreation', 'HeaderLength', 'CntWord', 'CntTitleWord', 'CntSentance', 'TempInc', 'NormReputation', 
               'NormCntWord', 'NormCntTitleWord', 'NormCntSentance']
X_train = np.array(data_train[feature_set])
X_test = np.array(data_test[feature_set])
X_real_test = np.array(data_real_test[feature_set])
y_train = np.array(data_train.OpenStatus)
y_test = np.array(data_test.OpenStatus)

!!!


In [5]:
words = ['my', 'your', 'favorite', 'best', 'random', 'python', 'up', 'down', 'change', 'qsort', 'pascal', ':=', 'the',
         'and', 'to', 'have', 'look', 'this', 'from', 'in' 'for', 'if', 'else', 'import', 'def', 'void', 'int', 'double', 
         'back', 'time', 'now', 'just', 'more', 'way', 'to', 'try', 'call', 'help', 'work', 'window', 'step', 'tree', 'name', 
         'path', 'both', 'sure', 'last', 'next', '\n', '    ', 'an', 'data', 'real', 'tree', 'host', 'unix', 'win', 'xp', 
         'phone', ]
def add_counts_words(X, data, words):
    for w in words:
        y = data.Title.apply(lambda x: x.lower().count(w))
        y = np.array(y)[:, np.newaxis]
        X = np.hstack((X, y))
        y = data.BodyMarkdown.apply(lambda x: x.lower().count(w))
        y = np.array(y)[:, np.newaxis]
        X = np.hstack((X, y))
    return X

In [6]:
X_train = add_counts_words(X_train, data, words)
X_test = add_counts_words(X_test, datatest, words)
X_real_test = add_counts_words(X_real_test, datarealtest, words)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_real_test = scaler.transform(X_real_test)

###KNN

###Деревья

In [7]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=25, min_samples_leaf=11, n_estimators=170)
clf.fit(X_train, y_train)
print(roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1]))
#0.775223476161
#0.800654597744

0.800188757893


## Bag-of-words

In [8]:
import nltk
import nltk.stem

In [9]:
docs = list(np.concatenate((np.array(data_train.BodyMarkdown), np.array(data_real_test.BodyMarkdown))))
tags = list(np.concatenate((np.array(data_train.Tags), np.array(data_real_test.Tags))))
docs_2 = list(np.concatenate((np.array(data_train.FirstIndent), np.array(data_real_test.FirstIndent))))

In [10]:
vect = TfidfVectorizer(min_df=10)#, stop_words='english')
vect.fit(docs)
X_train_tfidf = vect.transform(data_train.BodyMarkdown)
X_test_tfidf = vect.transform(data_real_test.BodyMarkdown)
print('!!!')

!!!


In [11]:
vect_tags = TfidfVectorizer(min_df=5)#, stop_words='english')
vect_tags.fit(tags)
X_train_tfidf_tags = vect_tags.transform(data_train.Tags)
X_test_tfidf_tags = vect_tags.transform(data_real_test.Tags)
print('!!!')

!!!


In [12]:
vect_2 = TfidfVectorizer(min_df=10)#, stop_words='english')
vect_2.fit(docs_2)
X_train_tfidf_2 = vect_2.transform(data_train.FirstIndent)
X_test_tfidf_2 = vect_2.transform(data_real_test.FirstIndent)
print('!!!')

!!!


In [13]:
vect_chars = TfidfVectorizer(min_df=50, analyzer='char')
vect_chars.fit(docs)
X_train_tfidf_chars = vect_chars.transform(data_train.BodyMarkdown)
X_test_tfidf_chars = vect_chars.transform(data_real_test.BodyMarkdown)
print('!!!')

!!!


In [14]:
vect_chars_tags = TfidfVectorizer(min_df=10, analyzer='char')
vect_chars_tags.fit(tags)
X_train_tfidf_chars_tags = vect_chars_tags.transform(data_train.Tags)
X_test_tfidf_chars_tags = vect_chars_tags.transform(data_real_test.Tags)
print('!!!')

!!!


In [17]:
print('!!!')
import scipy.sparse as sps
X_train_2 = sps.hstack((X_train, X_train_tfidf))
X_test_2 = sps.hstack((X_real_test, X_test_tfidf))

X_train_2 = sps.hstack((X_train_2, X_train_tfidf_tags))
X_test_2 = sps.hstack((X_test_2, X_test_tfidf_tags))

X_train_2 = sps.hstack((X_train_2, X_train_tfidf_2))
X_test_2 = sps.hstack((X_test_2, X_test_tfidf_2))

X_train_2 = sps.hstack((X_train_2, X_train_tfidf_chars))
X_test_2 = sps.hstack((X_test_2, X_test_tfidf_chars))

X_train_2 = sps.hstack((X_train_2, X_train_tfidf_chars_tags))
X_test_2 = sps.hstack((X_test_2, X_test_tfidf_chars_tags))

!!!


#### Линейные модели

In [18]:
from sklearn.linear_model import LogisticRegression
clf_linear = LogisticRegression()
clf_linear.fit(X_train_2, y_train)
#print(roc_auc_score(y_test, clf_linear.predict_proba(X_test_2)[:, 1]))
#0.864784895409
#0.87036472842 without stop_words
#0.870540386559 without stop_words + new words(count)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

#### но мы потратили время не зря!

In [13]:
p1 = clf_linear.predict_proba(X_test_2)[:, 1]
p2 = clf.predict_proba(X_test)[:, 1]
for alpha in np.arange(0, 1.01, 0.01):
    p = roc_auc_score(y_test, alpha * p1 + (1 - alpha) * p2)
    if p > 0.8758:
        print(alpha, p)

0.63 0.875840143714
0.64 0.875912756658
0.65 0.875970910645
0.66 0.876009822196
0.67 0.876034352479
0.68 0.876041777416
0.69 0.876031825707
0.7 0.876012171391
0.71 0.875978170336
0.72 0.875928081303
0.73 0.875866665572


In [19]:
print('!!!')
p1 = clf_linear.predict_proba(X_test_2)[:, 1]
p2 = clf.predict_proba(X_real_test)[:, 1]

!!!


In [34]:
alpha = 0.60
ans = alpha * p1 + (1 - alpha) * p2

In [35]:
res = datarealtest.copy()
res['OpenStatus'] = ans

In [36]:
res.to_csv('result.csv', mode='w', index=False, columns=['PostId', 'OpenStatus'])